### 手写数字识别
对于输入图像，预测图中的数字是0到9中的哪个的问题（10类别分类问题）。

#### MNIST数据集
**load_mnist** 函数以“**(训练图像，训练标签),(测试图像，测试标签)**”的形式返回读入的MNIST数据。
load_mnist(normalize=True,flatten=True,one_hot_label=True) 可以设置3个参数。
* normalize设置是否将输入图像正规化为0.0\~1.0的值。<br>如果将该参数设置为False，则输入图像的像素会保持原来的0\~255。<br>如果设置为True，函数内部会进行转换，将图像的各个像素值除以255，使得数据的值在0.0\~1.0的范围内（正规化）。
* flatten设置是否展开输入图像（变成一维数组）。<br>如果将该参数设置为False，则输入图像为1x28x28的三维数组；<br>若设置为True，则输入图像会保存为由784个元素构成的一维数组。
* one_hot_label设置是否将标签保存为one-hot表示，就像[0,0,1,0,0,0,0,0,0,0]。<br>当one_hot_label为False时，只是像7，2这样简单保存正确解标签；<br>当one_hot_label为True时，标签则保存为one-hot表示。

#### 显示MNITS图像(训练图像的第一张)

In [1]:
import sys, os
sys.path.append(os.pardir) # 把父目录加入到sys.path(Python的搜索模块的路径集)中，从而可以导入父目录下的任何目录
import numpy as np
from dataset.mnist import load_mnist
from PIL import Image # 图像的显示使用PIL(Python Image Library)模块

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))# 把保存为Numpy数组的图像数据转换为PIL用的数据对象
    pil_img.show()
    
(x_train, t_train), (x_test,t_test) = load_mnist(flatten=True, normalize=False) # flatten为True，表示展开输入图像（变成一位数组），如果为False，则输入图像为1*28*28的三维数组

img = x_train[0]    # 第一张训练图像
label = t_train[0]  # 第一个训练标签

print(label) # 5

print(img.shape)          # (784,)
img = img.reshape(28, 28) # 把图像的形状变成原来的尺寸
print(img.shape)          # (28, 28)

img_show(img)

5
(784,)
(28, 28)


#### 神经网络的推理处理

In [46]:
# neuralnet_mnist.py
import numpy as np
import pickle

def sigmoid(x):
    return 1 / (1+np.exp(-x))

def softmax(a): # 输出层的激活函数
    c = np.max(a)
    exp_a = np.exp(a-c) # 溢出对策
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

In [53]:
def get_data(): # 从数据集中读取数据
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False) # 读取数据集
    return x_test, t_test

def init_network(): # 读入samle_weight.pkl中的权重参数
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    return network

def predict(network, x): # 
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)
    return y

In [54]:
# 实现神经网络的推理处理
x, t = get_data()
network = init_network()

accuracy_cnt = 0 # 识别精度，即能在多大程度上正确分类
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y) # 获取概率最高的元素的索引
    if p == t[i]:
        accuracy_cnt += 1

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352


#### 批处理

In [56]:
x, _ = get_data()
network = init_network()
W1, W2, W3 = network['W1'], network['W2'], network['W3']

x.shape

(10000, 784)

In [57]:
x[0].shape

(784,)

In [58]:
W1.shape

(784, 50)

In [59]:
W2.shape

(50, 100)

In [60]:
W3.shape

(100, 10)

通过np.argmax()获取值最大的元素的索引。
* 如果给定了参数 axis=0,代表沿着第0维，即列方向，找到值最大的元素的索引。
* 如果给定了参数 axis=1,代表沿着第1维，即行方向，找到值最大的元素的索引。

In [66]:
x = np.array([[0.1, 0.8, 0.1], [0.3, 0.1, 0.6], [0.2, 0.5, 0.3], [0.8, 0.1, 0.1]])
y = np.argmax(x, axis=1) # 沿着第1维，即行方向，找到值最大的元素的索引
print(y)

[1 2 1 0]


In [63]:
# 基于批处理实现代码
x, t = get_data()
network = init_network()

batch_size = 100 # 批数量
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])

print("Accuracy:" + str(float(accuracy_cnt / len(x))))

Accuracy:0.9352


### 小节


* 神经网络中的激活函数使用平滑变化的 sigmoid 函数或者 ReLU 函数。
* 通过巧妙地使用 NumPy 多维数组，可以高效地实现神经网络。
* 机器学习的问题大体上可以分为回归问题和分类问题。
* 关于输出层的激活函数，回归问题中一般用恒等函数，分类问题中一般用 softmax 函数。
* 分类问题中，输出层的神经元的数量设置为要分类的类别数。
* 输入数据的集合称为批。通过以批为单位进行推理处理，能够实现高效的运算。